In [1]:
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos


import time
start_time = time.time()


#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99


#???
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')

#Импорт таблиц номинального режима и частичной нагрузки
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

#Состав газов в номинальном режиме и в частичной нагрузке
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])


#???
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,
                                              prop.REFPROP_p_t,
                                              prop.REFPROP_p_h,
                                              prop.REFPROP_p_s,
                                              prop.REFPROP_p_q,
                                              prop.REFPROP_t_q,
                                              RP=RP)





PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
print(PEN)





iter1=50
Qq = n.zeros(iter1)
for j in range(iter1):
    iter2=50
    Qqq = n.zeros(iter2)
    Ddd = n.zeros(iter2)
    for i in range(iter2):
        PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',KPD, calctolerance,gas0,gas1,water,
                   calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        Qqq[i] = PEVD[8]+IVD[8]
        Ddd[i] = water_streams.at['PEVD-DROS','G']
        if i>0:
            if abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) < calctolerance and abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]) < calctolerance:
                break;
    EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',KPD, calctolerance,gas0,gas1,water,
                   calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
    water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    Qq[j] = PEVD[8]+IVD[8]+EVD[8]
    if j>0:
        if abs((Qq[j-1]-Qq[j])/Qq[j-1]) < calctolerance:
            break;
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
print(water_streams)

# iter1=50
# Qq = n.zeros(iter1)
# for j in range(iter1):
#     iter2=50
#     Qqq = n.zeros(iter2)
#     Ddd = n.zeros(iter2)
#     for i in range(iter2):
#         PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',KPD, calctolerance,gas0,gas1,water,
#                    calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#         gas_streams.loc['PPND-IND','T':'G'] = [PPND[0],PPND[1],PPND[2],PPND[3]]
#         water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
    
#         IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#         gas_streams.loc['IND-GPK','T':'G'] = [IND[0],IND[1],IND[2],IND[3]]
#         water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
    
#         water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
#         Qqq[i] = PPND[8]+IND[8]
#         Ddd[i] = water_streams.at['PPND-TURB','G']
#         if i>0:
#             print("--- %s сек. ---" % round((time.time() - start_time)))
#             print(abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) , abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]))
#             if abs((Qqq[i-1]-Qqq[i])/Qqq[i-1]) < calctolerance and abs((Ddd[i-1]-Ddd[i])/Ddd[i-1]) < calctolerance:
#                 break;
#     water_streams.loc['GPK-IND':'X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
    
#     GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
#     gas_streams.loc['GPK-out','T':'G'] = [GPK[0],GPK[1],GPK[2],GPK[3]]
#     water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]
    
#     Qq[j] = GPK[8]+IND[8]+PPND[8]
#     if j>0:
#         if abs((Qq[j-1]-Qq[j])/Qq[j-1]) < calctolerance:
#             break;




print("--- %s сек. ---" % round((time.time() - start_time)))

[26.850000000000023, 8.3412, -9999.99, 61.51, 0.0]
                    T         P            H          G
PEVD-DROS  496.959461  8.012386  3391.805379  60.715862
IVD-PEVD   297.938037  8.341200  2753.507839  60.715862
EVD-IVD    297.938037  8.341200  1375.194327  60.715862
PEN-EVD    166.400000  8.341200   707.655323  60.715862
BND-PEN           NaN  0.692500          NaN  61.510000
PPND-TURB  212.300000  0.687000  2873.213987  15.510000
IND-PPND   164.512241  0.692500  2762.297432  15.510000
GPK-IND    164.000000  0.692500   692.882238  15.510000
X-GPK       60.000000  0.692500   251.745121  77.020000
--- 7 сек. ---


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,936.681566,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,456.173544,0.1,860.833337,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,308.683794,0.1,697.102746,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.394570,0.1,617.793172,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,234.600000,0.1,616.940134,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.900000,0.1,554.284799,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,108.900000,0.1,483.533964,516.2,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
water_streams

,T,P,H,G
PEVD-DROS,496.959461,8.012386,3391.805379,60.715862
IVD-PEVD,297.938037,8.341200,2753.507839,60.715862
EVD-IVD,297.938037,8.341200,1375.194327,60.715862
PEN-EVD,166.400000,8.341200,707.655323,60.715862
BND-PEN,NaN,0.692500,NaN,61.510000
PPND-TURB,212.300000,0.687000,2873.213987,15.510000
IND-PPND,164.512241,0.692500,2762.297432,15.510000
GPK-IND,164.000000,0.692500,692.882238,15.510000
X-GPK,60.000000,0.692500,251.745121,77.020000
